In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import json
import random
import pandas as pd
import numpy as np
import torch
from torchvision import transforms
from typing import List, Set
from datetime import datetime
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

from deep_miml.models import Average, Attention
from deep_miml.train import train_miml_model, test_multi_instance_model
from deep_miml.utils import precision_recall_helper, get_avg_batch_precision_recall_at_k, timing
import torchvision.transforms as transforms
import torch.optim as optim
from pathlib import Path
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.enabled=False
torch.backends.cudnn.deterministic=True

/home/mridula/Projects/MIML/notebooks


In [3]:
class YelpDataset:
    def __init__(self, business_df, image_df, image_folder, transform):
        self.business_df = business_df
        self.image_df = image_df
        self.transform = transform
        self.image_folder = image_folder
        
    
    def __getitem__(self, i: int):
        b_id = self.business_df.iloc[i]['business_id']
        imgs = list(self.image_df[self.image_df['business_id'] == b_id]['photo_id'])
        
        imgs = [self.transform(Image.open(self.image_folder+f"{img}.jpg")) for img in imgs]
        
        labels = torch.FloatTensor(self.business_df.iloc[i,1:])
        
        return imgs, labels
        
    def __len__(self):
        return self.business_df.shape[0]
     
        
def collate_fn(batch, input_size=224):
    img_lists, bag_labels = zip(*batch)

    imgs = [img for img_list in img_lists for img in img_list]

    sizes = torch.LongTensor([len(img_list) for img_list in img_lists])

    bag_labels = torch.stack(bag_labels)
    

    if len(imgs) != 0:
        imgs = torch.stack(imgs)
    else:
        imgs = torch.zeros((0, 3, input_size, input_size)).float()

    return imgs, sizes, bag_labels
        

In [4]:
# images_folder_path = '../../WIML/yelp_data/website_data/yelp_photos/photos/'
# final_business_df = pd.read_csv('../../WIML/yelp_data/website_data/final_business_data.csv')
# final_images_df = pd.read_csv('../../WIML/yelp_data/website_data/final_images_data.csv')

# print(f'Shape of business_df {final_business_df.shape}')
# print(f'Shape of images_df {final_images_df.shape}')

Shape of business_df (23137, 9)
Shape of images_df (139443, 2)


In [5]:
# # Save the results for reproducability 
# full_business_list = set(final_business_df.business_id)

# business_split_list = {}

# business_split_list['test'] = random.sample(full_business_list,500)

# business_split_list['val'] = random.sample(
#     full_business_list.difference(business_split_list['test']),
#     1000)

# business_split_list['train'] = list(full_business_list.difference(
#     business_split_list['test']+business_split_list['val']))




In [6]:
# # Saving the train-test-val split to json file 
# import json

# with open('../../WIML/yelp_data/website_data/train_test_val_split.json', 'w') as fp:
#     json.dump(business_split_list, fp)

In [7]:
# # Select a maximum of n photos per restaurant
# # https://stackoverflow.com/questions/22472213/python-random-selection-per-group

# def limit_image_count(obj, size = 30, replace= False):
#     if len(obj.photo_id) <= size:
#         return obj
#     else:
#         return obj.loc[np.random.choice(obj.index, size, replace),:]


In [8]:
# subset_images_df = final_images_df.groupby('business_id',as_index=False).apply(limit_image_count)
# subset_images_df.to_csv('../../WIML/yelp_data/website_data/subset_images_data.csv', index = False)

In [4]:
# subset_images_df.groupby(['business_id'])['photo_id'].count().\
# reset_index().sort_values(by =['photo_id'], ascending = False)

# READING SAVED DATA

In [4]:
final_business_df = pd.read_csv('../../WIML/yelp_data/website_data/final_business_data.csv')
subset_images_df = pd.read_csv('../../WIML/yelp_data/website_data/subset_images_data.csv')
with open('../../WIML/yelp_data/website_data/train_test_val_split.json') as f:
    business_split_list = json.load(f)

images_folder_path = '../../WIML/yelp_data/website_data/yelp_photos/photos/'

In [12]:
ls -al ../../WIML/yelp_data/website_data/

total 7300956
drwxrwxr-x 4 mridula mridula       4096 Oct 30 15:20 ./
drwxr-xr-x 5 mridula mridula       4096 Oct 24 08:12 ../
-rw-rw-r-- 1 mridula mridula    1272663 Oct 27 15:56 final_business_data.csv
-rw-rw-r-- 1 mridula mridula    6414399 Oct 27 15:56 final_images_data.csv
-rw-r--r-- 1 mridula mridula    5800047 Jan 17  2020 list
-rw-rw-r-- 1 mridula mridula    5699467 Nov 15 12:40 subset_images_data.csv
-rw-rw-r-- 1 mridula mridula     601592 Nov 15 12:40 train_test_val_split.json
drwxrwxr-x 2 mridula mridula       4096 Oct 23 18:24 yelp_dataset/
drwxrwxr-x 3 mridula mridula       4096 Oct 23 18:17 yelp_photos/
-rw-r--r-- 1 mridula mridula 7456358400 Oct 23 18:07 yelp_photos.tar


In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}




image_datasets = {}

image_datasets['train'] = YelpDataset(
    business_df = final_business_df[
        final_business_df.business_id.isin(business_split_list['train'])],
    image_df = subset_images_df,
    image_folder = images_folder_path,
    transform = data_transforms['train']
)

image_datasets['val'] = YelpDataset(
    business_df= final_business_df[final_business_df.business_id.isin(business_split_list['val'])],
    image_df= subset_images_df,
    image_folder = images_folder_path,
    transform = data_transforms['val']
)

image_datasets['test'] = YelpDataset(
    business_df= final_business_df[final_business_df.business_id.isin(business_split_list['test'])],
    image_df= subset_images_df,
    image_folder = images_folder_path,
    transform = data_transforms['test']
)

# print( f'len of train dataset {train_dataset.__len__()}')
# print( f'len of val dataset {val_dataset.__len__()}')
# train_dataset.__getitem__(0)

# DataLoaders

In [6]:
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x],
                                                    batch_size= 4,
                                                    shuffle=(x == 'train'),
                                                    num_workers= 12,
                                                    collate_fn=collate_fn)
                    for x in ['train', 'val','test']}

print('DataLoaders ready ... \n')


DataLoaders ready ... 



In [8]:
dataloaders_dict['train'].__len__()

5410

In [9]:
dataloaders_dict['val'].__len__()

250

In [10]:
dataloaders_dict['test'].__len__()

125

# Training

In [15]:
def train_and_save(model_type,
                   model_name,
                   lr,
                   device,
                   num_categories,
                   num_epochs= 1,
                   early_stopping= True,
                   save_folder = '/home/mridula/Projects/MIML/temp',
                   patience =5, use_pretrained=True):

    if model_type == 'avg':
        model_ft = Average(num_classes = num_categories,model_name= model_name)
    elif model_type == 'attention':
        model_ft = Attention(num_classes = num_categories, model_name= model_name)

    else:
        print("Enter valid model type")
        exit(0)


        # Send the model to GPU
    model_ft = model_ft.to(device)

        # Observe that all parameters are being optimized
    optimizer_ft = optim.Adam(model_ft.parameters(), lr= lr)

    # You should pass logits to nn.BCEwithLogitsLoss and probabilities (using "sigmoid") to nn.BCELoss.
    # Using BCEWithLogitsLoss because it is more stable 
    criterion = torch.nn.BCEWithLogitsLoss(weight=None, reduction='mean')

        # Train and evaluate
    model_ft = train_miml_model(model=model_ft,
                                model_name=model_name,
                                model_type=model_type,
                                use_pretrained=use_pretrained,
                                device = device,
                                dataloaders=dataloaders_dict,
                                criterion=criterion,
                                optimizer=optimizer_ft,
                                save_folder= save_folder ,
                                lr=lr,
                                num_epochs= num_epochs,
                                early_stopping = early_stopping,
                                patience = patience
                                )

    torch.save(model_ft.state_dict(),
        Path.cwd().joinpath(save_folder,"yelp_{}_{}_pretrained_{}.pt".format(model_name, model_type, use_pretrained)))
    
    return model_ft


In [13]:
# Path.cwd().joinpath('/home/mridula/Projects/MIML/temp', 'hello')

In [16]:

num_categories = 8
lr = 0.0001
use_pretrained = True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# ResNet18

In [16]:
# Average model
resnet18_avg = train_and_save(
    model_type = 'avg',
    model_name = 'resnet18',
    lr = 0.001,
    device = device,
    num_categories= num_categories,
    num_epochs= 50,
    early_stopping= True,
    save_folder = '/home/mridula/Projects/MIML/temp',
    patience =5,
    use_pretrained=True
)

  0%|          | 0/5410 [00:00<?, ?it/s]

cuda:0
Epoch 0/49
----------


 19%|█▊        | 1001/5410 [02:36<12:14,  6.00it/s]


Running avg Loss at batch iter                                                                 1000 is                                                                 0.5101544723808765


 37%|███▋      | 2000/5410 [05:09<10:25,  5.45it/s]

...


Running avg Loss at batch iter                                                                 5000 is                                                                 0.4536869696378708


  0%|          | 0/250 [00:00<?, ?it/s]

train Loss: 0.4572

Train -- epoch average precision at k = 1, 2, 3, 4, 5, 6: [0.95938078 0.94849815 0.93308272 0.91207255 0.85134935 0.79393207]
Train -- epoch average recall at k = 1, 2, 3, 4, 5, 6: [0.18356839 0.36242652 0.53213031 0.6897976  0.79541359 0.88331701]



100%|██████████| 250/250 [00:13<00:00, 17.90it/s]

val Loss: 0.5580

Validation -- epoch average precision at k = 1, 2, 3, 4, 5, 6: [0.954   0.944   0.92769 0.91275 0.8512  0.79586]


Validation -- epoch average recall at k = 1, 2, 3, 4, 5, 6: [0.18225 0.36117 0.52921 0.69107 0.79418 0.88355]



In [18]:
%%time
# TESTING
result_resnet18_avg = test_multi_instance_model(resnet18_avg, device, dataloaders_dict['test'])

100%|██████████| 125/125 [00:07<00:00, 17.35it/s]

CPU times: user 6.39 s, sys: 432 ms, total: 6.82 s
Wall time: 7.21 s


In [19]:
result_resnet18_avg

{'precision_at': {1: 0.966, 2: 0.949, 3: 0.935, 4: 0.914, 5: 0.857, 6: 0.798},
 'recall_at': {1: 0.185, 2: 0.362, 3: 0.531, 4: 0.688, 5: 0.797, 6: 0.884}}

In [ ]:
# attention model
resnet18_attn = train_and_save(
    model_type = 'attention', 
    model_name = 'resnet18', 
    lr = 0.001, 
    device = device,
    num_categories= num_categories,
    num_epochs= 50,
    early_stopping= True,
    save_folder = '/home/mridula/Projects/MIML/temp',
    patience =5, 
    use_pretrained=True)

  0%|          | 0/5410 [00:00<?, ?it/s]

cuda:0
Epoch 0/49
----------


 18%|█▊        | 1000/5410 [02:49<09:56,  7.39it/s]


Running avg Loss at batch iter                                                                 1000 is                                                                 0.5113341208100319


 37%|███▋      | 2001/5410 [05:35<10:52,  5.22it/s]

...

 19%|█▊        | 1001/5410 [02:42<09:28,  7.76it/s]


Running avg Loss at batch iter                                                                 1000 is                                                                 0.44927272835373877


 37%|███▋      | 2001/5410 [05:31<08:45,  6.49it/s]


Running avg Loss at batch iter                                                                 2000 is                                                                 0.45237813124060633


 49%|████▉     | 2646/5410 [07:17<05:35,  8.25it/s]

In [25]:
result_resnet18_attn = test_multi_instance_model(resnet18_attn, device, dataloaders_dict['test'])
result_resnet18_attn

100%|██████████| 125/125 [00:07<00:00, 16.06it/s]


{'precision_at': {1: 0.956, 2: 0.953, 3: 0.943, 4: 0.924, 5: 0.86, 6: 0.802},
 'recall_at': {1: 0.183, 2: 0.364, 3: 0.538, 4: 0.696, 5: 0.799, 6: 0.888}}

# Resnet34


In [26]:
resnet34_avg = train_and_save(
    model_type = 'avg',
    model_name = 'resnet34',
    lr = 0.001,
    device = device,
    num_categories= num_categories,
    num_epochs= 50,
    early_stopping= True,
    save_folder = '/home/mridula/Projects/MIML/temp',
    patience =5,
    use_pretrained=True
)

  0%|          | 0/5410 [00:00<?, ?it/s]

cuda:0
Epoch 0/49
----------


 18%|█▊        | 1000/5410 [04:17<19:05,  3.85it/s]


Running avg Loss at batch iter                                                                 1000 is                                                                 0.5164081510603428


 37%|███▋      | 2000/5410 [08:45<13:45,  4.13it/s]

...


Running avg Loss at batch iter                                                                 5000 is                                                                 0.47701073083281514


  0%|          | 0/250 [00:00<?, ?it/s]

train Loss: 0.4814

Train -- epoch average precision at k = 1, 2, 3, 4, 5, 6: [0.95083179 0.94078096 0.92736599 0.90719732 0.8416451  0.78426433]
Train -- epoch average recall at k = 1, 2, 3, 4, 5, 6: [0.1812768  0.35832671 0.52724261 0.68506747 0.7851354  0.87167606]



100%|██████████| 250/250 [00:23<00:00, 10.59it/s]


val Loss: 0.6046

Validation -- epoch average precision at k = 1, 2, 3, 4, 5, 6: [0.954   0.9405  0.92699 0.90625 0.843   0.79011]


Validation -- epoch average recall at k = 1, 2, 3, 4, 5, 6: [0.18197 0.35802 0.52563 0.68348 0.78555 0.8771 ]



In [27]:
result_resnet34_avg = test_multi_instance_model(resnet34_avg, device, dataloaders_dict['test'])
result_resnet34_avg

100%|██████████| 125/125 [00:11<00:00, 10.51it/s]


{'precision_at': {1: 0.948, 2: 0.939, 3: 0.932, 4: 0.91, 5: 0.849, 6: 0.79},
 'recall_at': {1: 0.18, 2: 0.357, 3: 0.528, 4: 0.684, 5: 0.788, 6: 0.874}}

In [28]:
resnet34_attention = train_and_save(
    model_type = 'attention',
    model_name = 'resnet34',
    lr = 0.001,
    device = device,
    num_categories= num_categories,
    num_epochs= 50,
    early_stopping= True,
    save_folder = '/home/mridula/Projects/MIML/temp',
    patience =5,
    use_pretrained=True
)

  0%|          | 0/5410 [00:00<?, ?it/s]

cuda:0
Epoch 0/49
----------


 18%|█▊        | 1000/5410 [04:32<29:17,  2.51it/s]


Running avg Loss at batch iter                                                                 1000 is                                                                 0.5047597855329513


 37%|███▋      | 2001/5410 [09:16<13:10,  4.31it/s]

...


Running avg Loss at batch iter                                                                 5000 is                                                                 0.4464716542512178


  0%|          | 0/250 [00:00<?, ?it/s]

train Loss: 0.4536

Train -- epoch average precision at k = 1, 2, 3, 4, 5, 6: [0.95928835 0.95224122 0.93681423 0.91544593 0.85308688 0.79377819]
Train -- epoch average recall at k = 1, 2, 3, 4, 5, 6: [0.18370749 0.36460675 0.5351585  0.69283965 0.79738586 0.88329113]



100%|██████████| 250/250 [00:24<00:00, 10.10it/s]


val Loss: 0.4471

Validation -- epoch average precision at k = 1, 2, 3, 4, 5, 6: [0.957   0.9475  0.93836 0.9175  0.8546  0.79615]


Validation -- epoch average recall at k = 1, 2, 3, 4, 5, 6: [0.18258 0.36255 0.53618 0.6939  0.79817 0.88421]



In [29]:
result_resnet34_attention = test_multi_instance_model(resnet34_attention, device, dataloaders_dict['test'])
result_resnet34_attention



100%|██████████| 125/125 [00:12<00:00, 10.00it/s]


{'precision_at': {1: 0.948, 2: 0.953, 3: 0.941, 4: 0.917, 5: 0.856, 6: 0.796},
 'recall_at': {1: 0.181, 2: 0.363, 3: 0.536, 4: 0.689, 5: 0.796, 6: 0.882}}

# ResNet50

In [31]:
resnet50_avg = train_and_save(
    model_type = 'avg',
    model_name = 'resnet50',
    lr = 0.001,
    device = device,
    num_categories= num_categories,
    num_epochs= 50,
    early_stopping= True,
    save_folder = '/home/mridula/Projects/MIML/temp',
    patience =5,
    use_pretrained=True
)

  0%|          | 0/5410 [00:00<?, ?it/s]

cuda:0
Epoch 0/49
----------


 18%|█▊        | 1000/5410 [06:12<17:28,  4.21it/s]


Running avg Loss at batch iter                                                                 1000 is                                                                 0.5076859662830829


 37%|███▋      | 2001/5410 [12:30<17:54,  3.17it/s]

...


Running avg Loss at batch iter                                                                 5000 is                                                                 0.4803627085983753


  0%|          | 0/250 [00:00<?, ?it/s]

train Loss: 0.4856

Train -- epoch average precision at k = 1, 2, 3, 4, 5, 6: [0.94861368 0.93900185 0.92770009 0.90608826 0.83974122 0.78174677]
Train -- epoch average recall at k = 1, 2, 3, 4, 5, 6: [0.18082717 0.35762616 0.52747643 0.68408364 0.78327495 0.86879898]



100%|██████████| 250/250 [00:32<00:00,  7.77it/s]


val Loss: 0.4745

Validation -- epoch average precision at k = 1, 2, 3, 4, 5, 6: [0.952   0.937   0.92699 0.909   0.8402  0.78622]


Validation -- epoch average recall at k = 1, 2, 3, 4, 5, 6: [0.18179 0.35695 0.52685 0.68605 0.78266 0.87217]



In [33]:
result_resnet50_avg = test_multi_instance_model(resnet50_avg, device, dataloaders_dict['test'])
result_resnet50_avg

100%|██████████| 125/125 [00:16<00:00,  7.77it/s]


{'precision_at': {1: 0.94, 2: 0.934, 3: 0.932, 4: 0.916, 5: 0.846, 6: 0.784},
 'recall_at': {1: 0.178, 2: 0.354, 3: 0.528, 4: 0.688, 5: 0.784, 6: 0.866}}

In [17]:
resnet50_attention = train_and_save(
    model_type = 'attention',
    model_name = 'resnet50',
    lr = 0.001,
    device = device,
    num_categories= num_categories,
    num_epochs= 50,
    early_stopping= True,
    save_folder = '/home/mridula/Projects/MIML/temp',
    patience =5,
    use_pretrained=True
)

  0%|          | 0/5410 [00:00<?, ?it/s]

cuda:0
Epoch 0/49
----------


 18%|█▊        | 1000/5410 [06:33<28:08,  2.61it/s] 


Running avg Loss at batch iter                                                                 1000 is                                                                 0.5022626719474792


 37%|███▋      | 2000/5410 [13:11<23:09,  2.45it/s]

...


Running avg Loss at batch iter                                                                 5000 is                                                                 0.4428081227764487


  0%|          | 0/250 [00:00<?, ?it/s]

train Loss: 0.4452

Train -- epoch average precision at k = 1, 2, 3, 4, 5, 6: [0.96418669 0.95598429 0.93947551 0.91670518 0.85551756 0.79569224]
Train -- epoch average recall at k = 1, 2, 3, 4, 5, 6: [0.18500323 0.36655915 0.5375915  0.69454436 0.80014048 0.88568854]



100%|██████████| 250/250 [00:33<00:00,  7.40it/s]


val Loss: 0.4376

Validation -- epoch average precision at k = 1, 2, 3, 4, 5, 6: [0.965   0.9585  0.94167 0.92025 0.8604  0.79952]


Validation -- epoch average recall at k = 1, 2, 3, 4, 5, 6: [0.18466 0.36783 0.53823 0.69701 0.80342 0.88856]



In [18]:
torch.cuda.empty_cache()

# Results

In [ ]:
model_results = {}

for mn in ['resnet18', 'resnet34', 'resnet50']:
    for mt in ['avg', 'attention']:
        model = torch.load(f'/home/mridula/Projects/MIML/temp/yelp_{mn}_{mt}_pretrained_True.pt')
        model_results['mn'+ '_'+ mt] = test_multi_instance_model(model, device, dataloaders_dict['test'])
        